In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.makedirs("/root/.kaggle/", exist_ok=True)

import shutil
shutil.copy("/kaggle/input/kaggle/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)




In [2]:
!kaggle competitions download -c icaif-24-finance-rag-challenge


import zipfile
with zipfile.ZipFile("icaif-24-finance-rag-challenge.zip", "r") as zip_ref:
    zip_ref.extractall("data")

 99%|█████████████████████████████████████▊| 19.0M/19.1M [00:01<00:00, 19.1MB/s]
100%|██████████████████████████████████████| 19.1M/19.1M [00:01<00:00, 11.4MB/s]


In [3]:
import os
import pandas as pd
folders = [ele for ele in os.listdir("./data") if ele.split(".")[-1] == "jsonl"]

datasets = {folder : pd.read_json("./data/"+folder+"/corpus.jsonl", lines = True) for folder in folders if folder.find("corpus") != -1}
queries = {folder : pd.read_json("./data/"+folder+"/queries.jsonl", lines = True) for folder in folders if folder.find("queries") != -1}

In [4]:
!pip install faiss-gpu
!pip install sentence_transformers
!pip install rank_bm25
!pip install num2words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 7.0 MB/s eta 0:00:00


In [5]:
import os
from typing import List, Dict, Any, Set
import numpy as np
import faiss
import json
from tqdm import tqdm
import torch
import torch.cuda.amp
from sentence_transformers import SentenceTransformer, CrossEncoder
from dataclasses import dataclass
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer

@dataclass
class SearchResult:
    id: str
    title: str
    text: str
    score: float
    keywords: List[str] = None

class EnhancedHybridRAG:
    def __init__(
        self, 
        encoder_model_name: str = 'BAAI/bge-large-en-v1.5',
        reranker_model_name: str = 'BAAI/bge-reranker-large',
        index_path: str = 'faiss_index',
        metadata_path: str = 'document_metadata.json',
        alpha: float = 0.6,
        beta: float = 0.2,
        num_workers: int = 4
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.encoder = SentenceTransformer(encoder_model_name).to(self.device)
        self.reranker = CrossEncoder(reranker_model_name, device=self.device)
        
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.index = None
        self.metadata = []
        self.gpu_index = None
        self.document_texts = {}
        self.alpha = alpha
        self.beta = beta
        self.num_workers = num_workers
        
        # Initialize text processing
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)
        nltk.download('wordnet', quiet=True)
        self.stop_words = set(stopwords.words('english'))
        self.tfidf = TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 2),
            max_features=10000
        )
        
        # Initialize search components
        self.bm25 = None
        self.tokenized_corpus = None
        self.keyword_index = {}
        self.document_keywords = {}

    def extract_keywords(self, text: str) -> Set[str]:
        """Extract important keywords from text"""
        # Clean and tokenize
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        words = word_tokenize(text)
        
        # Remove stopwords and short words
        words = [w for w in words if w not in self.stop_words and len(w) > 2]
        
        # Get word frequencies
        word_freq = Counter(words)
        
        # Return top keywords
        return set(dict(word_freq.most_common(10)).keys())

    def preprocess_text(self, text: str) -> List[str]:
        """Enhanced text preprocessing"""
        # Clean text
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        # Tokenize
        tokens = word_tokenize(text)
        # Remove stopwords and short words
        tokens = [t for t in tokens if t not in self.stop_words and len(t) > 2]
        return tokens

    def encode_documents(self, documents: List[Dict[str, Any]], batch_size: int = 32):
        print("Encoding documents...")
        dimension = self.encoder.get_sentence_embedding_dimension()

        # Build TF-IDF index
        print("Building TF-IDF index...")
        doc_texts = [doc['text'] for doc in documents]
        self.tfidf.fit(doc_texts)

        # Extract keywords for all documents
        print("Extracting keywords...")
        for doc in tqdm(documents, desc="Processing keywords"):
            keywords = self.extract_keywords(doc['text'])
            self.document_keywords[doc['id']] = list(keywords)

            # Build keyword index
            for keyword in keywords:
                if keyword not in self.keyword_index:
                    self.keyword_index[keyword] = set()
                self.keyword_index[keyword].add(doc['id'])

        # Initialize FAISS index
        n_docs = len(documents)
        n_clusters = min(int(np.sqrt(n_docs)), max(1, n_docs // 4))

        # Create CPU index first
        if n_docs < 1000:
            self.index = faiss.IndexFlatL2(dimension)
        else:
            quantizer = faiss.IndexFlatL2(dimension)
            self.index = faiss.IndexIVFFlat(quantizer, dimension, n_clusters)

        # Process documents first to get vectors for training
        print("Processing documents...")
        train_vectors = []
        tokenized_corpus = []

        for doc in tqdm(documents, desc="Processing documents"):
            text = f"passage: {doc['text']}"
            tokenized_corpus.append(self.preprocess_text(doc['text']))

            with torch.no_grad():
                embedding = self.encoder.encode(text, convert_to_tensor=True).cpu().numpy()
            train_vectors.append(embedding)
            self.document_texts[doc['id']] = doc['text']

        train_vectors = np.array(train_vectors).astype('float32')

        # Initialize GPU resources and transfer index
        if torch.cuda.is_available():
            print("Moving index to GPU...")
            res = faiss.StandardGpuResources()

            if isinstance(self.index, faiss.IndexFlatL2):
                self.gpu_index = faiss.GpuIndexFlatL2(res, dimension)
            else:
                config = faiss.GpuIndexIVFFlatConfig()
                config.useFloat16 = True
                self.gpu_index = faiss.GpuIndexIVFFlat(
                    res, 
                    dimension,
                    n_clusters,
                    faiss.METRIC_L2
                )
                # Train IVF index
                print("Training IVF index...")
                self.gpu_index.train(train_vectors)
        else:
            print("No GPU available, using CPU index...")
            self.gpu_index = self.index
            if isinstance(self.index, faiss.IndexIVFFlat):
                self.index.train(train_vectors)

        # Initialize BM25
        self.bm25 = BM25Okapi(tokenized_corpus)
        self.tokenized_corpus = tokenized_corpus

        # Batch add vectors to index
        print("Adding vectors to index...")
        for i in tqdm(range(0, len(documents), batch_size), desc="Indexing"):
            batch = documents[i:i+batch_size]
            texts = [f"passage: {doc['text']}" for doc in batch]

            with torch.no_grad():
                embeddings = self.encoder.encode(texts, convert_to_tensor=True).cpu().numpy()

            self.gpu_index.add(embeddings)

            for doc in batch:
                self.metadata.append({
                    'id': doc['id'],
                    'title': doc['title'],
                    'text': doc['text'],
                    'keywords': list(self.document_keywords[doc['id']])
                })

        # Save index and metadata
        print("Saving index and metadata...")
        if torch.cuda.is_available():
            self.index = faiss.index_gpu_to_cpu(self.gpu_index)
        else:
            self.index = self.gpu_index

        faiss.write_index(self.index, self.index_path)

        # Save metadata with proper serialization
        serializable_corpus = [list(tokens) for tokens in self.tokenized_corpus]
        with open(self.metadata_path, 'w') as f:
            json.dump({
                'metadata': self.metadata,
                'document_texts': self.document_texts,
                'tokenized_corpus': serializable_corpus,
                'document_keywords': self.document_keywords,
                'keyword_index': {k: list(v) for k, v in self.keyword_index.items()}
            }, f)

        print("Indexing completed successfully.")
            
    def load_index(self):
        if os.path.exists(self.index_path) and os.path.exists(self.metadata_path):
            self.index = faiss.read_index(self.index_path)
            
            if torch.cuda.is_available():
                res = faiss.StandardGpuResources()
                if isinstance(self.index, faiss.IndexFlatL2):
                    self.gpu_index = faiss.GpuIndexFlatL2(res, self.index.d)
                    self.gpu_index.copyFrom(self.index)
                elif isinstance(self.index, faiss.IndexIVFFlat):
                    config = faiss.GpuIndexIVFFlatConfig()
                    config.useFloat16 = True
                    self.gpu_index = faiss.GpuIndexIVFFlat(
                        res, 
                        self.index.d,
                        self.index.nlist,
                        faiss.METRIC_L2
                    )
                    self.gpu_index.copyFrom(self.index)
            else:
                self.gpu_index = self.index
            
            with open(self.metadata_path, 'r') as f:
                data = json.load(f)
                self.metadata = data['metadata']
                self.document_texts = data['document_texts']
                self.tokenized_corpus = [list(tokens) for tokens in data['tokenized_corpus']]
                self.document_keywords = data['document_keywords']
                self.keyword_index = {k: set(v) for k, v in data['keyword_index'].items()}
                self.bm25 = BM25Okapi(self.tokenized_corpus)
            
            print(f"Loaded index with {self.gpu_index.ntotal} vectors.")
        else:
            print("Index files not found. Please encode documents first.")

    def get_keyword_matches(self, query: str, doc_id: str) -> float:
        """Calculate keyword matching score"""
        query_keywords = self.extract_keywords(query)
        doc_keywords = set(self.document_keywords.get(doc_id, []))
        if not doc_keywords:
            return 0.0
        
        matches = len(query_keywords.intersection(doc_keywords))
        return matches / len(query_keywords) if query_keywords else 0.0

    def batch_search(
        self, 
        queries: List[str], 
        k: int = 10,
        rerank_k: int = 25,
        rerank_batch_size: int = 64
    ) -> List[List[SearchResult]]:
        if self.gpu_index is None:
            self.load_index()

        all_results = []

        # Optimize GPU memory for P100
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.memory.set_per_process_memory_fraction(0.9)
            torch.backends.cudnn.benchmark = True
            torch.backends.cuda.matmul.allow_tf32 = True

        for query_idx, query in enumerate(queries, 1):
            print(f"\nProcessing query {query_idx}/{len(queries)}: {query}")

            query_text = f"query: {query}"

            with torch.no_grad():
                query_vector = self.encoder.encode(
                    query_text,
                    convert_to_tensor=True,
                    batch_size=1,
                    show_progress_bar=False
                ).cpu().numpy().astype('float32').reshape(1, -1)

                query_tokens = self.preprocess_text(query)
                bm25_scores = self.bm25.get_scores(query_tokens)
                max_bm25 = max(bm25_scores) + 1e-6
                query_keywords = self.extract_keywords(query)

            if isinstance(self.index, faiss.IndexIVFFlat):
                self.gpu_index.nprobe = min(8, self.gpu_index.nlist)
            dense_distances, dense_indices = self.gpu_index.search(query_vector, rerank_k)

            results = []
            seen_ids = set()

            for idx, distance in zip(dense_indices[0], dense_distances[0]):
                if idx != -1 and idx < len(self.metadata):
                    doc_id = self.metadata[idx]['id']
                    if doc_id not in seen_ids:
                        seen_ids.add(doc_id)

                        # Get document text from metadata
                        doc_text = self.metadata[idx]['text']  # Fixed: Get text from metadata

                        dense_score = 1 / (1 + distance)
                        sparse_score = bm25_scores[idx] / max_bm25
                        keyword_score = self.get_keyword_matches(query, doc_id)

                        combined_score = (
                            self.alpha * dense_score + 
                            (1 - self.alpha - self.beta) * sparse_score +
                            self.beta * keyword_score
                        )

                        results.append({
                            'id': doc_id,
                            'title': self.metadata[idx]['title'],
                            'text': doc_text,  # Now doc_text is defined
                            'score': combined_score,
                            'keywords': self.document_keywords[doc_id]
                        })

            if results:
                results = sorted(results, key=lambda x: x['score'], reverse=True)[:rerank_k]
                rerank_pairs = [(query, res['text']) for res in results]

                with torch.cuda.amp.autocast():
                    with torch.no_grad():
                        rerank_scores = self.reranker.predict(
                            rerank_pairs,
                            batch_size=rerank_batch_size,
                            show_progress_bar=False
                        )

                final_results = [
                    SearchResult(
                        id=res['id'],
                        title=res['title'],
                        text=res['text'],
                        score=float(score),
                        keywords=res['keywords']
                    )
                    for res, score in zip(results, rerank_scores)
                ]

                final_results = sorted(final_results, key=lambda x: x.score, reverse=True)[:k]
                all_results.append(final_results)
                print(f"Found {len(final_results)} results")
            else:
                all_results.append([])

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return all_results
    


    
documents = [
    {'id': str(1), 'title': f'Document my name', 'text': 'My Name is rahul verma'},
    {'id': str(2), 'title': f'Document current city', 'text': 'I currently live in chandigarh punjab'},
    {'id': str(3), 'title': f'Document pet name', 'text': 'I have a cat name rishi'},
    {'id': str(110), 'title': f'financial document', 'text': 'The following tables present the recorded investment by portfolio segment and by class, excluding commercial financing receivables and other miscellaneous financing receivables at December 31, 2019 and 2018. Commercial financing receivables are excluded from the presentation of financing receivables by portfolio segment, as they are short term in nature and the current estimated risk of loss and resulting impact to the company’s financing results are not material. Write-offs of lease receivables and loan receivables were $16 million and $47 million, respectively, for the year ended December 31, 2019. Provisions for credit losses recorded for lease receivables and loan receivables were a release of $6 million and an addition of $2 million, respectively, for the year ended December 31, 2019. The average recorded investment of impaired leases and loans for Americas, EMEA and Asia Pacific was $138 million, $49 million and $45 million, respectively, for the year ended December 31, 2019. Both interest income recognized, and interest income recognized on a cash basis on impaired leases and loans were immaterial for the year ended December 31, 2019. ($ in millions) | | | | ---------------------------------------------------------- | -------- | ------ | ------------ | -------- At December 31, 2019: | Americas | EMEA | Asia Pacific | Total Recorded investment: | | | | Lease receivables | $ 3,419 | $1,186 | $ 963 | $ 5,567 Loan receivables | 6,726 | 3,901 | 2,395 | 13,022 Ending balance | $10,144 | $5,087 | $3,359 | $18,590 Recorded investment, collectively evaluated for impairment | $10,032 | $5,040 | $3,326 | $18,399 Recorded investment, individually evaluated for impairment | $ 112 | $ 47 | $ 32 | $ 191 Allowance for credit losses | | | | Beginning balance at January 1, 2019 | | | | Lease receivables | $ 53 | $ 22 | $ 24 | $ 99 Loan receivables | 105 | 43 | 32 | 179 Total | $ 158 | $ 65 | $ 56 | $ 279 Write-offs | (42) | (3) | (18) | (63) Recoveries | 1 | 0 | 1 | 2 Provision | 5 | (7) | (3) | (5) Other* | (1) | 0 | (1) | (2) Ending balance at December 31, 2019 | $ 120 | $ 54 | $ 36 | $ 210 Lease receivables | $ 33 | $ 23 | $ 16 | $ 72 Loan receivables | $ 88 | $ 31 | $ 20 | $ 138 Related allowance, collectively evaluated for impairment | $ 25 | $ 11 | $ 4 | $ 39 Related allowance, individually evaluated for impairment | $ 96 | $ 43 | $ 32 | $ 171'}
]

documents += [{'id': str(i), 'title': f'Document random {i}', 'text': f'This is a random document {i}'} for i in range(1, 100)]

rag = FinancialEnhancedRAG()


# # Encode and index documents
rag.encode_documents(documents)

# # Later, when you want to search:
# query = "What is the name of Rahul's pet"
# results = rag.search(query, k=1)

# print("Top relevant document:")
# for result in results:
#     print(f"ID: {result['id']}, Title: {result['title']}, Score: {result['score']:.4f}")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Indexing:  75%|███████▌  | 3/4 [00:01<00:00,  2.00it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Indexing: 100%|██████████| 4/4 [00:01<00:00,  2.09it/s]

Saving index and metadata...
Indexing completed successfully.


In [8]:
query = "What is Rahul's pet"
rag.batch_search(['What were the write offs in 2019', query, "What is Rahul's city", "Was there a name mentioned?"]
            )


Processing query 1/4: What were the write offs in 2019


/tmp/ipykernel_30/4146480938.py:333: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Found 25 results

Processing query 2/4: What is Rahul's pet
Found 25 results

Processing query 3/4: What is Rahul's city
Found 25 results

Processing query 4/4: Was there a name mentioned?
Found 25 results


[[], [], [], []]

In [27]:
def dataset_processor(documents, queries):
    documents = [
        {'id': row["_id"], 'title': row["title"], 'text': row["title"]+". "+row["text"]} for _, row in documents.iterrows()
    ]
    rag = FinancialEnhancedRAG()
    rag.encode_documents(documents)
    
    
    result = {"query_id":[], "corpus_id": []}
    output = rag.batch_search([query["text"] for _, query in queries.iterrows()])
    for _output, query in zip(output, queries.iterrows()):
        for _result in _output:
            result["query_id"].append(query[1]["_id"])
            result["corpus_id"].append(_result.id)
    del rag
    df = pd.DataFrame(result)
    return df 
        
    

In [17]:
datasets.keys()

dict_keys(['finder_corpus.jsonl', 'finqabench_corpus.jsonl', 'multiheirtt_corpus.jsonl', 'convfinqa_corpus.jsonl', 'financebench_corpus.jsonl', 'finqa_corpus.jsonl', 'tatqa_corpus.jsonl'])

In [28]:
for key in datasets:
    if key not in ['finder_corpus.jsonl', 'financebench_corpus.jsonl', 'finqabench_corpus.jsonl']:
        query_file = key.split("_")[0] + "_queries.jsonl"
        output_file = key.split("_")[0]+ ".tsv"
        output = dataset_processor(datasets[key], queries[query_file])
        output.to_csv(output_file, sep = "\t")    


Encoding documents...
Building TF-IDF index...
Extracting keywords...


Processing keywords: 100%|██████████| 2066/2066 [00:12<00:00, 169.50it/s]


Processing documents...


Processing documents:   0%|          | 0/2066 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing documents:  30%|███       | 627/2066 [00:58<02:13, 10.78it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing documents:  30%|███       | 627/2066 [00:59<02:15, 10.61it/s]


KeyboardInterrupt: 

In [2]:
import pandas as pd
import glob

# Path to the folder containing the TSV files to merge
folder_path = '/kaggle/input/new-results'

# Get all the TSV file paths in the folder
tsv_files = glob.glob(f"{folder_path}/*.tsv")

# Create an empty list to store dataframes
dataframes = []

# Read the first file with the header, and the rest without headers
for i, file in enumerate(tsv_files):
    if i == 0:
        df = pd.read_csv(file, sep='\t')  # Read the first file with the header
    else:
        df = pd.read_csv(file, sep='\t', header=None)  # Read the rest without the header
        df.columns = dataframes[0].columns  # Use the header from the first file
    dataframes.append(df)

# Concatenate all the dataframes
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe to a new TSV file
#merged_df.to_csv('merged_output.csv', index=False)

In [3]:
merged_df = merged_df[["query_id", "corpus_id"]]

In [4]:
merged_df.to_csv('merged_output_reiteration.csv', index=False)

In [21]:
!kaggle competitions submit -c icaif-24-finance-rag-challenge -f merged_output_reiteration.csv -m "new results"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████| 911k/911k [00:00<00:00, 1.50MB/s]
Successfully submitted to ACM-ICAIF '24 FinanceRAG Challenge